In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/encoded-df3/encoded_df3.csv


In [2]:
# Import dependencies 
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

plt.style.use('dark_background')

In [3]:
# Creating dataframe for beers and filtering for the US
encoded_df3 = pd.read_csv("../input/encoded-df3/encoded_df3.csv")
encoded_df3 ['style'] = le.fit_transform(encoded_df3 ['style'])
encoded_df3

,state_beer,style,availability,abv,types,taste
0,36,18,2,5.0,4,4.50
1,36,18,6,5.6,2,4.50
2,43,18,6,5.4,1,4.00
3,37,18,4,5.0,10,4.25
4,46,18,2,4.5,3,4.00
...,...,...,...,...,...,...
443995,4,36,6,5.8,3,4.50
443996,10,36,2,8.0,2,3.50
443997,4,36,2,11.0,10,5.00
443998,22,36,2,13.1,5,4.75


In [4]:
#Create a seperate df for y
style_df = encoded_df3.filter(['style'], axis=1)
style_df

,style
0,18
1,18
2,18
3,18
4,18
...,...
443995,36
443996,36
443997,36
443998,36


In [5]:
#Drop y from the main df
encoded_df3 = encoded_df3.drop(['style'], axis=1)
encoded_df3

,state_beer,availability,abv,types,taste
0,36,2,5.0,4,4.50
1,36,6,5.6,2,4.50
2,43,6,5.4,1,4.00
3,37,4,5.0,10,4.25
4,46,2,4.5,3,4.00
...,...,...,...,...,...
443995,4,6,5.8,3,4.50
443996,10,2,8.0,2,3.50
443997,4,2,11.0,10,5.00
443998,22,2,13.1,5,4.75


In [6]:
#Check the data types for each culumn
encoded_df3.dtypes

state_beer        int64
availability      int64
abv             float64
types             int64
taste           float64
dtype: object

In [7]:
#Scale the data
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [8]:
scaled_df = data_scaler.fit_transform(encoded_df3)
scaled_df

array([[ 0.88565368, -0.64351804, -0.80772653,  0.0723576 ,  1.08583817],
       [ 0.88565368,  1.21588562, -0.56203395, -0.8323034 ,  1.08583817],
       [ 1.36369417,  1.21588562, -0.64393148, -1.28463391,  0.27682337],
       ...,
       [-1.29967427, -0.64351804,  1.64919924,  2.78634061,  1.89485298],
       [-0.0704273 , -0.64351804,  2.50912325,  0.5246881 ,  1.49034558],
       [-0.54846779,  1.21588562,  1.23971161, -1.28463391,  1.08583817]])

In [9]:
# Define features set
X = scaled_df.copy()

# Define target vector
y = style_df.values

In [10]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [12]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [13]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [14]:
#calculate matrix and accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm)
acc_score = accuracy_score(y_test, predictions)

In [15]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Confusion Matrix")
print("Classification Report")
print(classification_report(y_test, predictions))

Accuracy: 71.02%
Confusion Matrix
Classification Report
              precision    recall  f1-score   support

           0       0.68      0.73      0.71      3651
           1       0.76      0.79      0.77      3689
           2       0.67      0.80      0.73      3668
           3       0.81      0.87      0.84      3638
           4       0.67      0.84      0.74      3584
           5       0.70      0.63      0.66      3683
           6       0.65      0.80      0.72      3681
           7       0.60      0.58      0.59      3685
           8       0.67      0.57      0.61      3671
           9       0.67      0.53      0.59      3747
          10       0.69      0.77      0.73      3637
          11       0.82      0.87      0.84      3610
          12       0.52      0.45      0.48      3701
          13       0.77      0.84      0.80      3724
          14       0.75      0.62      0.68      3687
          15       0.79      0.80      0.79      3593
          16       0.84  

In [16]:
train_predictions = model.predict(X_train)
print(classification_report(y_train, train_predictions))

              precision    recall  f1-score   support

           0       0.70      0.75      0.72      7449
           1       0.78      0.82      0.80      7411
           2       0.68      0.82      0.74      7432
           3       0.82      0.88      0.85      7462
           4       0.69      0.85      0.76      7516
           5       0.72      0.65      0.69      7417
           6       0.67      0.82      0.74      7419
           7       0.62      0.61      0.61      7415
           8       0.69      0.58      0.63      7429
           9       0.68      0.56      0.62      7353
          10       0.70      0.78      0.74      7463
          11       0.84      0.88      0.86      7490
          12       0.56      0.47      0.51      7399
          13       0.77      0.86      0.81      7376
          14       0.77      0.66      0.71      7413
          15       0.81      0.81      0.81      7507
          16       0.86      0.90      0.88      7459
          17       0.76    